In [6]:
!pip install scikit-learn pandas 
!pip install openai tiktoken backoff
!pip install pyarrow fastparquet

# 基于Embedding向量进行文本聚类

In [12]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd

def twenty_newsgroup_to_cvs():
    newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']

    targets = pd.DataFrame( newsgroups_train.target_names, columns=['title'])
    
    out = pd.merge(df, targets, left_on='target', right_index=True)
    out.to_csv('data/20_newsgroup.csv', index=False)

twenty_newsgroup_to_cvs()

In [13]:
import os,tiktoken, backoff

embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"
batch_size = 2000
max_tokens = 800

df = pd.read_csv('data/20_newsgroup.csv')
print("Number of rows before null filtering:", len(df))
df = df[df['text'].isnull() == False]
encoding = tiktoken.get_encoding(embedding_encoding)

df['n_tokens'] = df.text.apply(lambda x: len(encoding.encode(x)))
print("Number of rows before token number filtering:", len(df))
df = df[df.n_tokens<=max_tokens]
print("Number of rows data used:", len(df))

# df取前100行
df = df.head(100)



Number of rows before null filtering: 11314
Number of rows before token number filtering: 11096
Number of rows data used: 10481


In [ ]:
# 以下代码比较消耗 Token，你可以不运行
# @backoff.on_exception(backoff.expo, openai.error.RateLimitError)
# def get_embeddings_with_backoff(prompts, engine):
#     embeddings = []
#     for i in range(0, len(prompts), batch_size):
#         batch = prompts[i:i+batch_size]
#         embeddings += get_embeddings(list_of_text=batch, engine=engine)
#     return embeddings

# prompts = df.text.tolist()
# prompt_batches = [prompts[i:i+batch_size] for i in range(0, len(prompts), batch_size)]

# embeddings = []
# for batch in prompt_batches:
#     batch_embeddings = get_embeddings_with_backoff(prompts=batch, engine=embedding_model)
#     embeddings += batch_embeddings

# df["embedding"] = embeddings
# df.to_parquet("data/20_newsgroup_with_embedding.parquet", index=False)

In [14]:
import numpy as np
from sklearn.cluster import KMeans

embedding_df = pd.read_parquet("data/20_newsgroup_with_embedding.parquet")

print(embedding_df.head())

matrix = np.vstack(embedding_df.embedding.values)
num_of_clusters = 20

kmeans = KMeans(n_clusters=num_of_clusters, init="k-means++", n_init=10, random_state=42)
kmeans.fit(matrix)
labels = kmeans.labels_
embedding_df["cluster"] = labels

                                                text  target      title  \
0  I was wondering if anyone out there could enli...       7  rec.autos   
1  \nIt depends on your priorities.  A lot of peo...       7  rec.autos   
2  an excellent automatic can be found in the sub...       7  rec.autos   
3  : Ford and his automobile.  I need information...       7  rec.autos   
4  \nYo! Watch the attributions--I didn't say tha...       7  rec.autos   

   n_tokens                                          embedding  
0       121  [-0.0391300804913044, 0.013502633199095726, -0...  
1       108  [-0.0011249205563217402, -0.00376517535187304,...  
2       476  [-0.018259447067975998, -0.008410007692873478,...  
3        86  [-0.012589422054588795, 0.006539034191519022, ...  
4       130  [-0.0006192282889969647, -0.011226896196603775...  


In [15]:
# 统计每个cluster的数量
new_df = embedding_df.groupby('cluster')['cluster'].count().reset_index(name='count')

# 统计这个cluster里最多的分类的数量
title_count = embedding_df.groupby(['cluster', 'title']).size().reset_index(name='title_count')
first_titles = title_count.groupby('cluster').apply(lambda x: x.nlargest(1, columns=['title_count']))
first_titles = first_titles.reset_index(drop=True)
new_df = pd.merge(new_df, first_titles[['cluster', 'title', 'title_count']], on='cluster', how='left')
new_df = new_df.rename(columns={'title': 'rank1', 'title_count': 'rank1_count'})

# 统计这个cluster里第二多的分类的数量
second_titles = title_count[~title_count['title'].isin(first_titles['title'])]
second_titles = second_titles.groupby('cluster').apply(lambda x: x.nlargest(1, columns=['title_count']))
second_titles = second_titles.reset_index(drop=True)
new_df = pd.merge(new_df, second_titles[['cluster', 'title', 'title_count']], on='cluster', how='left')
new_df = new_df.rename(columns={'title': 'rank2', 'title_count': 'rank2_count'})
new_df['first_percentage'] = (new_df['rank1_count'] / new_df['count']).map(lambda x: '{:.2%}'.format(x))
# 将缺失值替换为 0
new_df.fillna(0, inplace=True)
# 输出结果
from IPython.display import display
display(new_df)

/tmp/ipykernel_207/3526204769.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  first_titles = title_count.groupby('cluster').apply(lambda x: x.nlargest(1, columns=['title_count']))
/tmp/ipykernel_207/3526204769.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  second_titles = second_titles.groupby('cluster').apply(lambda x: x.nlargest(1, columns=['title_count']))


,cluster,count,rank1,rank1_count,rank2,rank2_count,first_percentage
0,0,522,rec.autos,432,comp.sys.mac.hardware,6.0,82.76%
1,1,391,comp.sys.ibm.pc.hardware,101,comp.sys.mac.hardware,85.0,25.83%
2,2,1060,talk.politics.misc,129,talk.religion.misc,60.0,12.17%
3,3,381,rec.motorcycles,364,comp.sys.mac.hardware,1.0,95.54%
4,4,783,comp.sys.ibm.pc.hardware,323,comp.sys.mac.hardware,314.0,41.25%
5,5,659,soc.religion.christian,409,talk.religion.misc,151.0,62.06%
6,6,358,sci.crypt,345,comp.sys.mac.hardware,1.0,96.37%
7,7,84,comp.os.ms-windows.misc,8,comp.sys.mac.hardware,8.0,9.52%
8,8,477,rec.sport.hockey,461,0,0.0,96.65%
9,9,472,sci.space,403,comp.sys.mac.hardware,1.0,85.38%


# 使用提示语对文本进行总结

In [16]:
from openai import OpenAI

client = OpenAI()

items_per_cluster = 10
COMPLETIONS_MODEL = "gpt-3.5-turbo-instruct"

num_of_clusters = 5

for i in range(num_of_clusters):
    cluster_name = new_df[new_df.cluster == i].iloc[0].rank1
    print(f"Cluster {i}, Rank 1: {cluster_name}, Theme:", end=" ")

    content = "\n".join(
        embedding_df[embedding_df.cluster == i].text.sample(items_per_cluster, random_state=42).values
    )
    response = client.completions.create(
        model=COMPLETIONS_MODEL,
        prompt=f'''我们想要给下面的内容，分组成有意义的类别，以便我们可以对其进行总结。请根据下面这些内容的共同点，总结一个50个字以内的新闻组的名称。比如 “PC硬件”\n\n内容:\n"""\n{content}\n"""新闻组名称：''',
        temperature=0,
        max_tokens=100,
        top_p=1,
    )
    print(response.choices[0].text.replace("\n", ""))

Cluster 0, Rank 1: rec.autos, Theme: 汽车技术与安全
Cluster 1, Rank 1: comp.sys.ibm.pc.hardware, Theme: Macintosh Hardware and Software Discussions
Cluster 2, Rank 1: talk.politics.misc, Theme: 法律诉讼与税务纠纷
Cluster 3, Rank 1: rec.motorcycles, Theme: Motorcycle Safety and Dog Encounters
Cluster 4, Rank 1: comp.sys.ibm.pc.hardware, Theme: 计算机硬件升级与维护讨论组


In [17]:
items_per_cluster = 1
COMPLETIONS_MODEL = "gpt-3.5-turbo-instruct"

for i in range(num_of_clusters):
    cluster_name = new_df[new_df.cluster == i].iloc[0].rank1
    print(f"Cluster {i}, Rank 1: {cluster_name}, 抽样翻译:", end=" ")

    content = "\n".join(
        embedding_df[(embedding_df.cluster == i) & (embedding_df.n_tokens > 100)].text.sample(items_per_cluster, random_state=42).values
    )
    response = client.completions.create(
        model=COMPLETIONS_MODEL,
        prompt=f'''请把下面的内容翻译成中文\n\n内容:\n"""\n{content}\n"""翻译：''',
        temperature=0,
        max_tokens=2000,
        top_p=1,
    )
    print(response.choices[0].text.replace("\n", ""))

Cluster 0, Rank 1: rec.autos, 抽样翻译: 福特也在1983年尝试过这样做。我的1983年的Ranger皮卡的喇叭在转向灯杆的末端，而不是在方向盘的中心，这是上帝想要的位置。:-)当时我开过两辆不同的车（另一辆是1984年的凯美瑞），但从来没有习惯过按转向灯杆来鸣喇叭。唯一一次我做对了是在进行年度州政府要求的安全检查时！这不是福特最好的主意。
Cluster 1, Rank 1: comp.sys.ibm.pc.hardware, 抽样翻译: 我正在市场上寻找一台激光打印机。二手打印机可以，非苹果品牌的打印机也可以，但是无论我选择哪种打印机，都必须具备以下特点：必备特点：- PostScript- 300 dpi- 足够的内存来下载字体- 价格越便宜越好。我希望价格在1000美元以下。额外的特点：- 每分钟打印超过4页- SCSI输出接口用于字体盘- 占地面积小不关心的特点：- PostScript Level II- 网络功能（只是为了我自己，将来也是）- 彩色打印我知道有关Freedom of Press的信息，但我也听说它的打印速度非常慢。我可以接受每分钟打印4页，但如果比这还慢，我就不认为节省的价格值得。我很想听听别人对它的使用经验。我也非常喜欢激光打印机，我以前在上一份工作中使用过惠普的Deskjet打印机，对其质量并不满意。我承认我是激光打印机的偏执者。 :)我将使用这台打印机来排版我正在写的一本书的页面。页面将包括多种字体、PS图形、扫描的线稿和可能的灰度图片（还不确定）。质量不需要太好，但必须清晰可读。我看过的打印机：- 二手的LaserWriters：plus、NT、NTR。据我所知，只有NTR有一个用于字体盘的SCSI输出接口。是真的吗？- Personal LaserWriter（LS和NTR）。我可以享受苹果员工折扣（我在苹果的一个分支机构工作），所以我可以以合理的价格买到这些打印机。我听说LS的性能不好，有什么评论吗？我倾向于Personal NTR，因为它的占地面积小。- LaserWriter Select 300。我听说它没有PostScript，但我没有看到任何确切的信息。我曾经听说过一个“PostScript升级”的传闻。是真的吗？- 二手的惠普LaserJets。我在另一个平台上使用过II和IIP，它

# 利用文本摘要支持无限轮数的聊天

In [18]:
history = """User : 你是谁？
Assistant : 我是一个人工智能语言模型，可以回答各种问题，包括做菜的问题。

User : 请问鱼香肉丝怎么做？
Assistant : 鱼香肉丝的做法：

1. 猪里脊肉切丝，加入盐、淀粉、料酒搅拌均匀，腌制10分钟
2. 胡萝卜、青椒、木耳切丝备用
3. 热锅凉油，放入蒜末、姜末煸炒出香味，加入肉丝翻炒至变色
4. 加入胡萝卜、青椒、木耳翻炒，加入酱油、糖、醋、盐、味精、水淀粉搅拌均匀，倒入锅中翻炒均匀即可。

提示：鱼香肉丝的关键是调料的配比，酱油、糖、醋、盐、味精的比例要适合自己的口味。

User : 那蚝油牛肉呢？
Assistant : 蚝油牛肉的做法：

1. 将牛肉切成薄片，加入盐、淀粉、料酒腌制10分钟
2. 葱姜切末，蒜切片，青红椒切丝备用
3. 热锅凉油，放入葱姜末、蒜片煸炒出香味，加入牛肉翻炒至变色
4. 加入蚝油、酱油、糖、水淀粉调味，加入青红椒翻炒均匀即可。

提示：蚝油牛肉的关键是牛肉要切薄，翻炒时火候要快，保证牛肉口感鲜嫩。调味时，蚝油和酱油的比例也要适合自己的口味。
"""

def summarize(text, max_tokens=200):
    response = client.completions.create(
        model=COMPLETIONS_MODEL,
        prompt=text + "\n\n请总结一下上面User和Assistant聊了些什么：\n",
        max_tokens=max_tokens,
    )
    return response.choices[0].text

summarized = summarize(history)
print(summarized)


User询问了Assistant是谁，然后询问了鱼香肉丝和蚝油牛肉的做法。Assistant回答了两道菜的做法，并给出了一些关键提示。


In [19]:
!pip list

Package                        Version
------------------------------ --------------
aiofiles                       22.1.0
aiosqlite                      0.20.0
altair                         5.3.0
annotated-types                0.6.0
anyio                          4.3.0
argon2-cffi                    23.1.0
argon2-cffi-bindings           21.2.0
arrow                          1.3.0
astroid                        3.1.0
asttokens                      2.4.1
attrs                          23.2.0
autopep8                       2.0.4
Babel                          2.14.0
backoff                        2.2.1
beautifulsoup4                 4.12.3
bleach                         6.1.0
certifi                        2024.2.2
cffi                           1.16.0
charset-normalizer             3.3.2
click                          8.1.7
comm                           0.2.2
contourpy                      1.2.1
cramjam                        2.8.3
cycler                         0.12.1
debugpy        